In [1]:
import os
import shutil
import tempfile
import time
from typing import Any, Dict, List, Tuple, Union

import argparse
import joblib
from joblib import Parallel, delayed
import numpy as np
import pandas as pd 
import pickle as pkl
from termcolor import cprint
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm 
import wandb 

from logger import Logger
import utils.baseline_config as config
import utils.baseline_utils as baseline_utils
from utils.lstm_utils import ModelUtils, LSTMDataset
from prog_utils import * 
from models import * 
from lstm_prog_train_test import * 

use_cuda = torch.cuda.is_available()
if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
global_step = 0
best_loss = float("inf")
np.random.seed(100)

ROLLOUT_LENS = [1, 10, 30]

In [2]:
import sys
sys.argv = ['']
args = parse_arguments()
args.use_delta = True

In [3]:
data_dict = baseline_utils.get_data(args, 'none')
# add the program data to the data_dict
new_data_train = pd.read_pickle('Traj/train_prep.pkl')
new_data_val = pd.read_pickle('Traj/val_1_seg.pkl')
test_prep = pd.read_pickle('Traj/val_1_seg.pkl')
# new_data = pd.read_pickle('train_prep_para.pkl')
# test_prep = pd.read_pickle('train_prep_para.pkl')

data_dict['train_helpers'].CANDIDATE_CENTERLINES = new_data_train.CANDIDATE_CENTERLINES
data_dict['train_helpers']['PROG'] = new_data_train.PROG
data_dict['val_helpers'].CANDIDATE_CENTERLINES = new_data_val.CANDIDATE_CENTERLINES
data_dict['val_helpers']['PROG'] = new_data_val.PROG 


Loading Test data ...
Creating relative distances...
Test Size: 39472
Loading Train data ...
Creating relative distances...
Train Size: 39472
Loading Val data ...
Creating relative distances...
Val Size: 39472


In [4]:
new_data_val['CLS_RELATIVE'] = [[] for i in range(new_data_val['CANDIDATE_CENTERLINES'].shape[0])]
for i in range(new_data_val['CANDIDATE_CENTERLINES'].shape[0]):
    for j in range(len(new_data_val['CANDIDATE_CENTERLINES'][i])):
        # new_data_train['CANDIDATE_CENTERLINES'][i][j] = new_data_train['CANDIDATE_CENTERLINES'][i][j] + data_dict['train_helpers'].DELTA_REFERENCE[i]
        new_data_val['CLS_RELATIVE'][i].append(new_data_val['CANDIDATE_CENTERLINES'][i][j] - data_dict['val_helpers'].DELTA_REFERENCE[i])


In [6]:
new_data_val.to_pickle('Traj/val_1_seg.pkl')
# new_data_train = pd.read_pickle('Traj/train_prep.pkl')
# new_data_val.iloc[1]

In [2]:
data = pd.read_pickle('Traj/test_prep.pkl')

In [13]:
data.iloc[13]['PROG_PRED']

[(6, 15, 0.015488971024751663),
 (6, 15, 0.043969444930553436),
 (6, 15, 0.01836306042969227)]